In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from dotenv import load_dotenv
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [2]:
# Load the data into a Pandas DataFrame .
artist_df = pd.read_csv("Resources/artist_df.csv") 

# Display sample data
artist_df.head(10)

,id,name,image_url,isni,code2,hometown_city,verified,current_city,sp_followers,sp_popularity,...,popularity,rank,change,velocity,streak,created_at,peak_date,peak_rank,time_on_chart,rankStats
0,2762,Taylor Swift,https://share.chartmetric.com/artists/299/172/...,0000000078519858,US,NaN,True,NaN,114465831.0,100.0,...,100,1,0.0,0.000000,78,2024-07-01 08:58:49.864,2024-07-01,1,2273,"[{'date': '2024-06-18', 'rank': 1}, {'date': '..."
1,5596,Billie Eilish,https://i.scdn.co/image/ab67616100005174d8b998...,000000046748058X,US,Los Angeles,False,NaN,95726393.0,94.0,...,94,2,0.0,0.000000,78,2024-07-01 08:58:49.864,2019-06-28,1,2271,"[{'date': '2024-06-18', 'rank': 2}, {'date': '..."
2,3380,Drake,https://i.scdn.co/image/ab67616100005174429338...,000000012032246X,CA,Toronto,False,NaN,90019984.0,94.0,...,94,3,0.0,0.000000,78,2024-07-01 08:58:49.864,2021-11-16,1,2277,"[{'date': '2024-06-18', 'rank': 3}, {'date': '..."
3,3852,The Weeknd,https://i.scdn.co/image/ab6761610000e5eb214f3c...,0000000382556637,CA,NaN,False,NaN,85489663.0,93.0,...,93,4,0.0,0.000000,78,2024-07-01 08:58:49.864,2023-03-22,1,2265,"[{'date': '2024-06-18', 'rank': 5}, {'date': '..."
4,214945,Bad Bunny,https://i.scdn.co/image/ab6761610000e5eb9ad50e...,0000000466373221,PR,NaN,True,NaN,82987624.0,94.0,...,93,5,0.0,0.000000,78,2024-07-01 08:58:49.864,2023-03-20,1,2267,"[{'date': '2024-06-18', 'rank': 4}, {'date': '..."
5,3768,Kendrick Lamar,https://i.scdn.co/image/ab6761610000e5eb437b9e...,0000000362907030,US,Compton,False,NaN,30374768.0,92.0,...,92,6,0.0,0.000000,78,2024-07-01 08:58:49.864,2018-04-15,2,2261,"[{'date': '2024-06-18', 'rank': 6}, {'date': '..."
6,4215,Travis Scott,https://i.scdn.co/image/ab6761610000e5eb19c279...,0000000384091917,US,Houston,False,Houston,30348985.0,92.0,...,92,7,0.0,-0.142857,78,2024-07-01 08:58:49.864,2018-09-20,2,2273,"[{'date': '2024-06-18', 'rank': 7}, {'date': '..."
7,236,Eminem,https://i.scdn.co/image/ab6761610000e5eba00b11...,0000000368611919,US,St. Joseph,False,Detroit,86054535.0,91.0,...,91,8,0.0,0.142857,78,2024-07-01 08:58:49.864,2020-02-09,1,2274,"[{'date': '2024-06-18', 'rank': 9}, {'date': '..."
8,3963,Ariana Grande,https://i.scdn.co/image/ab6761610000e5eb40b5c0...,0000000059110272,US,NaN,False,Boca Raton,97308371.0,91.0,...,91,9,-2.0,-0.142857,78,2024-07-01 08:58:49.864,2020-12-02,1,2267,"[{'date': '2024-06-18', 'rank': 10}, {'date': ..."
9,135326,Post Malone,https://i.scdn.co/image/ab6761610000e5ebe17c0a...,0000000463129709,US,NaN,False,NaN,44358034.0,90.0,...,90,10,1.0,0.142857,78,2024-07-01 08:58:49.864,2020-01-14,1,2269,"[{'date': '2024-06-18', 'rank': 8}, {'date': '..."


In [3]:
artist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    200 non-null    int64  
 1   name                  200 non-null    object 
 2   image_url             200 non-null    object 
 3   isni                  137 non-null    object 
 4   code2                 199 non-null    object 
 5   hometown_city         87 non-null     object 
 6   verified              200 non-null    bool   
 7   current_city          57 non-null     object 
 8   sp_followers          199 non-null    float64
 9   sp_popularity         199 non-null    float64
 10  sp_monthly_listeners  199 non-null    float64
 11  deezer_fans           197 non-null    float64
 12  cm_artist_rank        199 non-null    float64
 13  cm_artist_score       199 non-null    float64
 14  tags                  199 non-null    object 
 15  spotify_artist_ids    2

In [4]:
artist_df.describe()

,id,sp_followers,sp_popularity,sp_monthly_listeners,deezer_fans,cm_artist_rank,cm_artist_score,popularity,rank,change,velocity,streak,peak_rank,time_on_chart
count,2.000000e+02,1.990000e+02,199.000000,1.990000e+02,1.970000e+02,199.000000,199.000000,200.000000,200.000000,197.000000,194.000000,200.000000,200.000000,200.000000
mean,9.106615e+05,2.120980e+07,83.587940,3.917335e+07,3.690390e+06,539.497487,193228.814070,83.500000,100.500000,2.228426,0.418998,68.000000,36.765000,1859.500000
std,2.281853e+06,2.214005e+07,3.668189,1.946266e+07,4.427857e+06,2932.438699,114397.603987,3.658816,57.879185,5.281234,2.102686,23.889612,36.512265,677.188149
min,1.900000e+01,1.533580e+05,79.000000,6.636940e+05,1.000000e+00,1.000000,3859.000000,79.000000,1.000000,-39.000000,-16.571429,1.000000,1.000000,2.000000
25%,2.940000e+03,7.281468e+06,81.000000,2.557317e+07,3.642980e+05,56.500000,116129.000000,81.000000,50.750000,2.000000,-0.571429,78.000000,7.000000,1634.750000
50%,5.348000e+03,1.436658e+07,82.000000,3.697712e+07,1.720242e+06,119.000000,170238.000000,82.000000,100.500000,3.000000,1.000000,78.000000,28.000000,2265.000000
75%,4.754602e+05,2.789524e+07,86.000000,5.010744e+07,5.758928e+06,234.500000,235514.000000,85.000000,150.250000,4.000000,1.714286,78.000000,52.250000,2273.000000
max,1.180197e+07,1.145540e+08,100.000000,1.060115e+08,2.309853e+07,27571.000000,646352.000000,100.000000,200.000000,6.000000,5.857143,78.000000,170.000000,2280.000000


In [5]:
artist_df.dtypes

id                        int64
name                     object
image_url                object
isni                     object
code2                    object
hometown_city            object
verified                   bool
current_city             object
sp_followers            float64
sp_popularity           float64
sp_monthly_listeners    float64
deezer_fans             float64
cm_artist_rank          float64
cm_artist_score         float64
tags                     object
spotify_artist_ids       object
itunes_artist_ids        object
deezer_artist_ids        object
amazon_artist_ids        object
timestp                  object
popularity                int64
rank                      int64
change                  float64
velocity                float64
streak                    int64
created_at               object
peak_date                object
peak_rank                 int64
time_on_chart             int64
rankStats                object
dtype: object

In [6]:
# Check for NaN values in the entire dataframe
nan_check = artist_df.isna().any().any()

if nan_check:
    print("There are NaN values in the dataframe.")
else:
    print("There are no NaN values in the dataframe.")

There are NaN values in the dataframe.


In [7]:
artist_df_without_na=artist_df.dropna()

In [8]:
# Check for NaN values in the entire dataframe
nan_check = artist_df_without_na.isna().any().any()

if nan_check:
    print("There are NaN values in the dataframe.")
else:
    print("There are no NaN values in the dataframe.")

There are no NaN values in the dataframe.


In [9]:
artist_df_without_na.columns

Index(['id', 'name', 'image_url', 'isni', 'code2', 'hometown_city', 'verified',
       'current_city', 'sp_followers', 'sp_popularity', 'sp_monthly_listeners',
       'deezer_fans', 'cm_artist_rank', 'cm_artist_score', 'tags',
       'spotify_artist_ids', 'itunes_artist_ids', 'deezer_artist_ids',
       'amazon_artist_ids', 'timestp', 'popularity', 'rank', 'change',
       'velocity', 'streak', 'created_at', 'peak_date', 'peak_rank',
       'time_on_chart', 'rankStats'],
      dtype='object')

In [10]:
# This is to initialize a LabelEncoder instance
le = LabelEncoder()

# This is to apply LabelEncoder to  each categorical column individually
artist_df_without_na.loc[:, 'name_encoded'] = le.fit_transform(artist_df_without_na['name'])
artist_df_without_na.loc[:, 'verified_encoded'] = le.fit_transform(artist_df_without_na['verified'])
artist_df_without_na.loc[:, 'tags_encoded'] = le.fit_transform(artist_df_without_na['tags'])
artist_df_without_na.loc[:, 'rankStats_encoded'] = le.fit_transform(artist_df_without_na['rankStats'])
artist_df_without_na.loc[:, 'timestp_encoded'] = le.fit_transform(artist_df_without_na['timestp'])
artist_df_without_na.loc[:, 'created_at_encoded'] = le.fit_transform(artist_df_without_na['created_at'])
artist_df_without_na.loc[:, 'peak_date_encoded'] = le.fit_transform(artist_df_without_na['peak_date'])

# This is to initialize a OneHotEncoder instance
ohe = OneHotEncoder()

# This is to apply OneHotEncoder to the columns labeled above
ohe_output = ohe.fit_transform(artist_df_without_na[['name_encoded', 'verified_encoded', \
                                          'tags_encoded', 'rankStats_encoded', \
                                            'timestp', 'created_at', 'peak_date']])


# Drop the original columns
artist_df_without_na.drop(['name', 'verified', 'tags', 'rankStats', 'timestp', 'created_at', 'peak_date'], axis=1, inplace=True)

C:\Users\phessou\AppData\Local\Temp\ipykernel_13056\1376342412.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artist_df_without_na.loc[:, 'name_encoded'] = le.fit_transform(artist_df_without_na['name'])
C:\Users\phessou\AppData\Local\Temp\ipykernel_13056\1376342412.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artist_df_without_na.loc[:, 'verified_encoded'] = le.fit_transform(artist_df_without_na['verified'])
C:\Users\phessou\AppData\Local\Temp\ipykernel_13056\1376342412.py:7: SettingWithCopyWarn

In [12]:
# Create the labels set `y` and features DataFrame `X`
# This is to copy the artists data into 'X'
X = artist_df_without_na.copy()

# This is to drop the column without significance to the study
X=X.drop(columns=['isni', 'code2', 'image_url', 'hometown_city', 'current_city',\
                     'sp_popularity', 'sp_followers', 'rank', 'spotify_artist_ids',\
                         'itunes_artist_ids', 'deezer_artist_ids',	'amazon_artist_ids', 'sp_popularity', 'sp_followers', 'rank'])

# This is to create the target DataFrames, 'y1,' 'y2,' and 'y3,' the depending variable crucial to our analysis
y1 = artist_df_without_na['sp_popularity']
y2 = artist_df_without_na['sp_followers']
y3 = artist_df_without_na['rank']


# Display 'y' to verify
y1.head()


6     92.0
7     91.0
10    90.0
11    90.0
15    90.0
Name: sp_popularity, dtype: float64

In [13]:
y2.head()

6     30348985.0
7     86054535.0
10     9788760.0
11    25824406.0
15     9820711.0
Name: sp_followers, dtype: float64

In [14]:
y3.head()

6      7
7      8
10    11
11    12
15    16
Name: rank, dtype: int64

In [15]:
X.head(10)

,id,sp_monthly_listeners,deezer_fans,cm_artist_rank,cm_artist_score,popularity,change,velocity,streak,peak_rank,time_on_chart,name_encoded,verified_encoded,tags_encoded,rankStats_encoded,timestp_encoded,created_at_encoded,peak_date_encoded
6,4215,66674476.0,2473569.0,33.0,297372.0,92,0.0,-0.142857,78,2,2273,33,0,5,34,0,0,4
7,236,84566034.0,17361566.0,3.0,551377.0,91,0.0,0.142857,78,1,2274,12,0,8,37,0,0,8
10,4550,84128121.0,521.0,20.0,337919.0,90,1.0,-0.714286,78,10,2273,29,0,18,19,0,0,20
11,1845,65903173.0,4173943.0,23.0,318463.0,90,0.0,0.142857,78,2,2275,18,0,8,2,0,0,10
15,212715,44708353.0,151815.0,106.0,177489.0,90,1.0,0.428571,78,6,1899,25,0,2,15,0,0,16
16,152776,43359471.0,720243.0,58.0,234694.0,90,1.0,0.428571,19,4,1923,13,0,12,7,0,0,12
23,439,82248589.0,16955455.0,13.0,375207.0,88,-7.0,-0.857143,78,4,2271,9,1,13,20,0,0,14
27,3501,74274248.0,11124510.0,19.0,355038.0,88,3.0,-0.428571,78,7,2269,6,0,19,21,0,0,14
38,2838,60782690.0,12070853.0,15.0,367228.0,86,3.0,0.428571,78,9,2264,4,0,22,22,0,0,18
43,127583,36977121.0,56389.0,126.0,164531.0,86,3.0,0.428571,78,13,1577,27,0,0,25,0,0,13


In [16]:
# Use train_test_split to separate the data
X_train, X_test, y1_train, y1_test = train_test_split(X, y1)
X_train, X_test, y2_train, y2_test = train_test_split(X, y2)
X_train, X_test, y3_train, y3_test = train_test_split(X, y3)

Model and Fit the Data to a Logistic Regression

In [17]:
# Create a `LogisticRegression` function and assign it 
# to a variable named `logistic_regression_model`.
logistic_regression_model = LogisticRegression()

In [18]:
X_train.head(10)

,id,sp_monthly_listeners,deezer_fans,cm_artist_rank,cm_artist_score,popularity,change,velocity,streak,peak_rank,time_on_chart,name_encoded,verified_encoded,tags_encoded,rankStats_encoded,timestp_encoded,created_at_encoded,peak_date_encoded
91,4018,32284127.0,4620913.0,164.0,141744.0,83,2.0,1.428571,78,36,2276,34,0,7,33,0,0,6
140,4408,22674108.0,238240.0,327.0,94580.0,81,5.0,1.714286,6,98,2265,8,1,3,3,0,0,1
51,3883,56701904.0,1.0,38.0,273410.0,85,3.0,0.857143,78,1,2222,14,0,23,23,0,0,7
198,152627,34739657.0,300484.0,174.0,137668.0,79,3.0,1.571429,78,14,1589,16,0,4,17,0,0,11
111,58,34293601.0,3396465.0,98.0,187628.0,82,4.0,1.857143,78,33,2266,15,0,8,1,0,0,12
23,439,82248589.0,16955455.0,13.0,375207.0,88,-7.0,-0.857143,78,4,2271,9,1,13,20,0,0,14
184,1464,20259985.0,1983348.0,247.0,112650.0,80,4.0,2.428571,78,118,2276,3,0,10,12,0,0,9
43,127583,36977121.0,56389.0,126.0,164531.0,86,3.0,0.428571,78,13,1577,27,0,0,25,0,0,13
170,2974,33468573.0,6762895.0,78.0,208561.0,80,4.0,2.142857,78,20,2274,37,0,8,9,0,0,0
54,1802,52311491.0,6919428.0,46.0,259891.0,85,3.0,-1.000000,78,2,2273,10,0,9,28,0,0,0


In [19]:
X_test.head(10)

,id,sp_monthly_listeners,deezer_fans,cm_artist_rank,cm_artist_score,popularity,change,velocity,streak,peak_rank,time_on_chart,name_encoded,verified_encoded,tags_encoded,rankStats_encoded,timestp_encoded,created_at_encoded,peak_date_encoded
11,1845,65903173.0,4173943.0,23.0,318463.0,90,0.0,0.142857,78,2,2275,18,0,8,2,0,0,10
38,2838,60782690.0,12070853.0,15.0,367228.0,86,3.0,0.428571,78,9,2264,4,0,22,22,0,0,18
57,748,43441138.0,10914411.0,59.0,234167.0,85,3.0,-1.285714,78,23,2273,21,0,25,30,0,0,0
109,5293,36500560.0,102.0,149.0,149768.0,82,4.0,2.000000,78,11,2271,35,0,17,36,0,0,3
90,102,35949280.0,7369981.0,103.0,182597.0,83,2.0,1.428571,78,24,2268,31,0,27,31,0,0,0
10,4550,84128121.0,521.0,20.0,337919.0,90,1.0,-0.714286,78,10,2273,29,0,18,19,0,0,20
27,3501,74274248.0,11124510.0,19.0,355038.0,88,3.0,-0.428571,78,7,2269,6,0,19,21,0,0,14
121,4796,49902303.0,5649997.0,110.0,175858.0,81,5.0,1.857143,78,25,2278,32,0,15,5,0,0,2
161,757,41007469.0,6931105.0,86.0,196434.0,80,5.0,2.000000,78,53,2246,36,0,21,6,0,0,0
52,217671,54673364.0,2120523.0,37.0,274265.0,85,3.0,0.714286,78,3,1754,11,0,17,27,0,0,9


In [20]:
# This is to create the StandardScaler instance
scaler = StandardScaler()

In [21]:
# Fit the Standard Scaler with the training data
X_train_scaled_n_fitted=scaler.fit(X_train)
X_test_scaled_n_fitted=scaler.fit(X_test)

In [22]:
# Scale the training data with 'transform'
X_train_scaled_n_transformed = scaler.transform(X_train)
X_test_scaled_n_transformed = scaler.transform(X_test)

In [23]:
# Train a Logistic Regression model and print the model score based on the first target 'y1'
logistic_regression_model = LogisticRegression(random_state=1)
logistic_regression_model.fit(X_train, y1_train)
print(f"Training Data Score: {logistic_regression_model.score(X_train_scaled_n_transformed, y1_train)}")

Training Data Score: 0.14285714285714285


c:\Users\phessou\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\phessou\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [24]:
# Train a Logistic Regression model and print the model score based on the second target 'y2'
logistic_regression_model = LogisticRegression(random_state=3)
logistic_regression_model.fit(X_train, y2_train)
logistic_regression_model.fit(X_train, y2_train)
print(f"Training Data Score: {logistic_regression_model.score(X_train_scaled_n_transformed, y2_train)}")

Training Data Score: 0.10714285714285714


c:\Users\phessou\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\phessou\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

In [25]:
# Train a Logistic Regression model and print the model score based on the third target 'y3'
logistic_regression_model = LogisticRegression(random_state=13)
logistic_regression_model.fit(X_train, y3_train)
logistic_regression_model.fit(X_train, y3_train)
print(f"Training Data Score: {logistic_regression_model.score(X_train_scaled_n_transformed, y3_train)}")

Training Data Score: 0.10714285714285714


c:\Users\phessou\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\phessou\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

In [26]:
# Make and save testing predictions with the saved logistic regression model using the test data
testing_predictions = logistic_regression_model.predict(X_test_scaled_n_transformed)
# Review the predictions
testing_predictions

c:\Users\phessou\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([ 52, 132, 132,  52, 132,  52, 132,  52, 132,  60], dtype=int64)

In [27]:
# Calculate the accuracy score by evaluating `y1_test` vs. `testing_predictions`.
accuracy = accuracy_score(y1_test, testing_predictions)
print("Accuracy Score:", accuracy)

Accuracy Score: 0.0


In [28]:
# Calculate the accuracy score by evaluating `y2_test` vs. `testing_predictions`.
accuracy = accuracy_score(y2_test, testing_predictions)
print("Accuracy Score:", accuracy)

Accuracy Score: 0.0


In [29]:
# Calculate the accuracy score by evaluating `y3_test` vs. `testing_predictions`.
accuracy = accuracy_score(y3_test, testing_predictions)
print("Accuracy Score:", accuracy)

Accuracy Score: 0.0


Random Forest Classifier Model

In [5]:
# Load the data into a Pandas DataFrame .
artist_tracks_df = pd.read_csv("../Team_9/Team_9//Resources/artist_tracks_df.csv") #index_col="coin_id")

# Display sample data
artist_tracks_df.head(10)

,id,isrc,artist_type,cm_artist,artist_names,code2s,artist_images,artist_covers,spotify_track_ids,spotify_album_ids,...,cm_audio_features.danceability,cm_audio_features.energy,cm_audio_features.instrumentalness,cm_audio_features.key,cm_audio_features.liveness,cm_audio_features.loudness,cm_audio_features.mode,cm_audio_features.speechiness,cm_audio_features.tempo,cm_audio_features.valence
0,11597863,DEHY11500279,main,"[2762, 215034, 602557, 48357]","['Taylor Swift', 'Max Martin', 'Shellback', 'B...","['US', 'US', None, None]",['https://share.chartmetric.com/artists/299/17...,"[None, None, None, None]","['7uUw9jra8yASVY96m23Pnn', '3ks57Gx7uPrHb1eyjM...","['4upWAaJn0tbtLFOX2YkeLG', '1Jk67VV9YnH7WUmNMB...",...,0,0,0,0,0,0,0,0,0,0
1,11597864,DEHY11500281,main,"[2762, 48357]","['Taylor Swift', 'B. Kudanowski']","['US', None]",['https://share.chartmetric.com/artists/299/17...,"[None, None]","['4bI8vJMklGft28G8DXnNr9', '2wbYQ5ouWIOMx19yGy...","['4upWAaJn0tbtLFOX2YkeLG', '1Jk67VV9YnH7WUmNMB...",...,0,0,0,0,0,0,0,0,0,0
2,12619780,GBCAD1300457,main,[2762],['Taylor Swift'],['US'],['https://share.chartmetric.com/artists/299/17...,[None],['1DEGUXEmD5uB7mWSOzIJJO'],['3O7uUWeS3H1D0EknhJstih'],...,0,0,0,0,0,0,0,0,0,0
3,13062145,GBUM70900575,main,"[2762, 116833]","['Taylor Swift', 'Digital Dog']","['US', None]",['https://share.chartmetric.com/artists/299/17...,"[None, None]","['0jjbxPKwUOGwITUzFfyYbK', '5XSm3THGdoV402U2JT...","['7CC3mcE01K9BndgyuDTjin', '4tRyMXoOGENdKI4tJG...",...,0,0,0,0,0,0,0,0,0,0
4,13062146,GBUM70900576,main,[2762],['Taylor Swift'],['US'],['https://share.chartmetric.com/artists/299/17...,[None],"['5UV1IMOt1oxIBj7FodKe9v', '4OzHfc5rHDDTIh5wyR...","['28Nq4AiAa0BW54KGWYFxot', '0ZBbJsW69ry5EE2hok...",...,0,0,0,0,0,0,0,0,0,0
5,13062147,GBUM70900580,main,"[2762, 678624]","['Taylor Swift', 'J.Stax']","['US', None]",['https://share.chartmetric.com/artists/299/17...,"[None, None]","['1Ay2JTezSnLTKYLt59k9Er', '4i0M7glXqIIRKPwbLt...","['78jOhaJZOtfqFP4Mh39xj0', '6tgMb6LEwb3yj7BdYy...",...,0,0,0,0,0,0,0,0,0,0
6,13062148,GBUM70900581,main,[2762],['Taylor Swift'],['US'],['https://share.chartmetric.com/artists/299/17...,[None],"['03pF3TQn62xUl74ww9yQxq', '0HUNbCRUodcJxbFQdr...","['2XxXXUWtVNnvQRkoenPnnb', '2s9WRNoIFC7gxnorPh...",...,0,0,0,0,0,0,0,0,0,0
7,13062149,GBUM70900582,main,[2762],['Taylor Swift'],['US'],['https://share.chartmetric.com/artists/299/17...,[None],['2qZSzX5VosoteeUJTpCoeG'],['6HlMlebpxACBKIJ8oj95F3'],...,0,0,0,0,0,0,0,0,0,0
8,13062151,GBUM70900586,main,[2762],['Taylor Swift'],['US'],['https://share.chartmetric.com/artists/299/17...,[None],"['5ru8NxVD4U5zH7wJtt1cE5', '7gfzBsM4xaMb8c2Wdp...","['2s9WRNoIFC7gxnorPhbQQA', '6tgMb6LEwb3yj7BdYy...",...,0,0,0,0,0,0,0,0,0,0
9,13063203,GBUM70908622,main,[2762],['Taylor Swift'],['US'],['https://share.chartmetric.com/artists/299/17...,[None],['48MQ4UlT16ETrIXLluZSFZ'],['52Zwt5P7dRlFeJZ83sRy8h'],...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Load the data into a Pandas DataFrame .
tracks_df = pd.read_csv("../Team_9/Team_9//Resources/tracks_df.csv") #index_col="coin_id")

# Display sample data
tracks_df.head(10)

,id,name,isrc,spotify_track_id,spotify_album_id,image_url,spotify_popularity,cm_track,spotify_artist_names,track_genre,...,chart_name,current_plays,code2,spotify,velocity,pre_rank,peak_rank,peak_date,time_on_chart,rankStats
0,140970481,Please Please Please,USUM72404101,5N3hjp1WNayUPZrA8kJmJP,5bBaoign62r1i7OV8w7mi9,https://i.scdn.co/image/ab67616d0000b2739968d9...,97,122336328,['Sabrina Carpenter'],"dance pop,electropop,pop,post-teen pop,viral pop",...,REGIONAL DAILY,NaN,US,140970481,0.000000,1,1,2024-06-10T00:00:00.000Z,25,"[{'plays': None, 'rank': 1, 'timestp': '2024-0..."
1,138255539,Not Like Us,USUG12400910,6AI3ezQ4o3HUoP6Dhudph3,5JjnoGJyOxfSZUZtk2rRwZ,https://i.scdn.co/image/ab67616d0000b2731ea0c6...,96,120189440,['Kendrick Lamar'],"conscious hip hop,hip hop,pop rap,rap,west coa...",...,REGIONAL DAILY,NaN,US,138255539,0.000000,2,1,2024-05-06T00:00:00.000Z,57,"[{'plays': None, 'rank': 2, 'timestp': '2024-0..."
2,139556669,A Bar Song (Tipsy),USUYG1541249,5fZJQrFKWQLb7FpJXZ1g7K,0DLvFVIfwt3OHdK9kGeEM3,https://i.scdn.co/image/ab67616d0000b27380d86d...,85,118969192,['Shaboozey'],pop rap,...,REGIONAL DAILY,NaN,US,139556669,0.428571,3,3,2024-06-29T00:00:00.000Z,31,"[{'plays': None, 'rank': 6, 'timestp': '2024-0..."
3,137338334,Espresso,USUM72403305,2qSkIjg1o9h3YT9RAgYN75,5quMTd5zeI9yW5UDua8wS4,https://i.scdn.co/image/ab67616d0000b273659cd4...,99,118981138,['Sabrina Carpenter'],"dance pop,electropop,pop,post-teen pop,viral pop",...,REGIONAL DAILY,NaN,US,137338334,-0.142857,4,1,2024-04-30T00:00:00.000Z,81,"[{'plays': None, 'rank': 3, 'timestp': '2024-0..."
4,138330270,I Had Some Help (feat. Morgan Wallen),USUM72404990,7221xIgOnuakPdLqT0F3nP,1woYXxyyxTQJ0E0AhZE6mj,https://is1-ssl.mzstatic.com/image/thumb/Music...,95,120372551,"['Post Malone', 'Morgan Wallen']","dfw rap,melodic rap,pop,rap,contemporary count...",...,REGIONAL DAILY,NaN,US,138330270,-0.142857,5,1,2024-05-11T00:00:00.000Z,52,"[{'plays': None, 'rank': 4, 'timestp': '2024-0..."
5,138482794,BIRDS OF A FEATHER,USUM72401994,6dOtVTDdiauQNBQEDOtlAB,7aJuG4TFXa2hmE4z1yxc3n,https://i.scdn.co/image/ab67616d0000b27371d62e...,98,120611262,['Billie Eilish'],"electropop,pop",...,REGIONAL DAILY,NaN,US,138482794,-0.142857,6,2,2024-06-05T00:00:00.000Z,45,"[{'plays': None, 'rank': 5, 'timestp': '2024-0..."
6,137031940,"Good Luck, Babe!",USUG12401967,0WbMK4wrZ1wFSty9F7FCgu,1WAjjRMfZjEXtB0lQrAw6Q,https://i.scdn.co/image/ab67616d0000b27391b4bc...,93,118460988,['Chappell Roan'],"indie electropop,springfield mo indie",...,REGIONAL DAILY,NaN,US,137031940,0.000000,7,5,2024-04-19T00:00:00.000Z,86,"[{'plays': None, 'rank': 7, 'timestp': '2024-0..."
7,140700554,MILLION DOLLAR BABY,QM24S2402528,5AJ9hqTS2wcFQCELCFRO7A,2Z1gnUf3nbn6DtwZSUIH54,https://i.scdn.co/image/ab67616d0000b27301c0cd...,93,119723065,['Tommy Richman'],NaN,...,REGIONAL DAILY,NaN,US,140700554,0.000000,8,3,2024-06-05T00:00:00.000Z,27,"[{'plays': None, 'rank': 8, 'timestp': '2024-0..."
8,130675615,HOT TO GO!,USUG12305911,4xdBrk0nFZaP54vvZj0yx7,0EiI8ylL0FmWWpgHVTsZjZ,https://is1-ssl.mzstatic.com/image/thumb/Music...,87,108232538,['Chappell Roan'],"indie electropop,springfield mo indie",...,REGIONAL DAILY,NaN,US,130675615,0.285714,10,9,2024-07-01T00:00:00.000Z,26,"[{'plays': None, 'rank': 11, 'timestp': '2024-..."
9,136864870,Too Sweet,IEACJ2300335,5Z0UnEtpLDQyYlWwgi8m9C,3P4SQqMMgjqfqVxgLwtYRk,https://i.scdn.co/image/ab67616d0000b2738d3151...,89,117691573,['Hozier'],"folk-pop,irish singer-songwriter,pop",...,REGIONAL DAILY,NaN,US,136864870,-0.142857,9,5,2024-04-24T00:00:00.000Z,69,"[{'plays': None, 'rank': 9, 'timestp': '2024-0..."


In [7]:
# Load the data into a Pandas DataFrame .
youtube_df = pd.read_csv("../Team_9/Team_9//Resources/youtube_df.csv") #index_col="coin_id")

# Display sample data
youtube_df.head(10)

,id,name,isrc,youtube_artist,image_url,cm_track,upload_date,license,album_names,youtube_artist_names,...,added_at,view_count,raw_data,velocity,pre_rank,peak_rank,peak_date,time_on_chart,rankStats,viewStats
0,fSMk5o9MWKM,Not Like Us,USUG12400910,/m/0g9x698,https://i.ytimg.com/vi/fSMk5o9MWKM/maxresdefau...,120189440.0,2024-05-05T07:36:05.000Z,NaN,['Not Like Us'],['Kendrick Lamar'],...,2024-06-27T00:00:00.000Z,12169849,NaN,0.000000,1.0,1,2024-06-20T00:00:00.000Z,2,"[{'rank': 1, 'timestp': '2024-06-20T00:00:00.0...","[{'views': 8965569, 'timestp': '2024-06-20T00:..."
1,t7bQwwqW-Hc,A Bar Song (Tipsy),USUYG1541249,/g/11b7_t9432,https://i.ytimg.com/vi/t7bQwwqW-Hc/maxresdefau...,118969192.0,2024-04-12T04:00:36.000Z,NaN,['A Bar Song (Tipsy)'],NaN,...,2024-06-27T00:00:00.000Z,7779581,NaN,0.000000,2.0,2,2024-06-20T00:00:00.000Z,11,"[{'rank': 2, 'timestp': '2024-06-20T00:00:00.0...","[{'views': 7735131, 'timestp': '2024-06-20T00:..."
2,bUX8MDNQda4,MILLION DOLLAR BABY,QM24S2402528,/g/11h2hbdm9p,https://i.ytimg.com/vi/bUX8MDNQda4/maxresdefau...,119723065.0,2024-05-03T04:02:07.000Z,NaN,['Million Dollar Baby - Single'],NaN,...,2024-06-27T00:00:00.000Z,6918255,NaN,0.000000,3.0,2,2024-05-30T00:00:00.000Z,5,"[{'rank': 3, 'timestp': '2024-06-20T00:00:00.0...","[{'views': 7513463, 'timestp': '2024-06-20T00:..."
3,GZ3zL7kT6_c,Lose Control (Live),USWB12302315,/g/11h763y_g9,https://i.ytimg.com/vi/GZ3zL7kT6_c/maxresdefau...,104296194.0,2023-06-23T04:00:12.000Z,NaN,['Lose Control'],NaN,...,2024-06-27T00:00:00.000Z,5319690,NaN,0.142857,5.0,1,2024-03-07T00:00:00.000Z,44,"[{'rank': 5, 'timestp': '2024-06-20T00:00:00.0...","[{'views': 5917639, 'timestp': '2024-06-20T00:..."
4,Oa_RSwwpPaA,Beautiful Things,USWB12307016,/g/11r_mnt170,https://i.ytimg.com/vi/Oa_RSwwpPaA/maxresdefau...,116167805.0,2024-01-18T23:00:13.000Z,NaN,['Beautiful Things'],NaN,...,2024-06-27T00:00:00.000Z,5193394,NaN,0.142857,6.0,1,2024-03-21T00:00:00.000Z,23,"[{'rank': 6, 'timestp': '2024-06-20T00:00:00.0...","[{'views': 5433504, 'timestp': '2024-06-20T00:..."
5,4QIZE708gJ4,I Had Some Help (feat. Morgan Wallen),USUM72404990,/g/11bw82cs0m,https://i.ytimg.com/vi/4QIZE708gJ4/maxresdefau...,120372551.0,2024-05-10T14:00:06.000Z,NaN,['I Had Some Help'],['Post Malone'],...,2024-06-27T00:00:00.000Z,5049040,NaN,0.285714,8.0,2,2024-05-16T00:00:00.000Z,7,"[{'rank': 8, 'timestp': '2024-06-20T00:00:00.0...","[{'views': 5114395, 'timestp': '2024-06-20T00:..."
6,22tVWwmTie8,Houdini,USUG12403398,/m/01vsgrn,https://i.ytimg.com/vi/22tVWwmTie8/maxresdefau...,121490756.0,2024-05-31T04:00:02.000Z,NaN,['Houdini'],['Eminem'],...,2024-06-27T00:00:00.000Z,4824661,NaN,-0.428571,4.0,1,2024-06-06T00:00:00.000Z,4,"[{'rank': 4, 'timestp': '2024-06-20T00:00:00.0...","[{'views': 6125175, 'timestp': '2024-06-20T00:..."
7,cF1Na4AIecM,Please Please Please,USUM72404101,/m/0lkzqr1,https://i.ytimg.com/vi/cF1Na4AIecM/maxresdefau...,122336328.0,2024-06-07T00:00:06.000Z,NaN,['Please Please Please'],['Sabrina Carpenter'],...,2024-06-27T00:00:00.000Z,4784037,NaN,-0.142857,7.0,3,2024-06-13T00:00:00.000Z,3,"[{'rank': 7, 'timestp': '2024-06-20T00:00:00.0...","[{'views': 5257589, 'timestp': '2024-06-20T00:..."
8,-V-sywKOCwQ,Si No Quieres No,US3DF2417042,/g/11gk34lb01,https://i.ytimg.com/vi/-V-sywKOCwQ/maxresdefau...,117171327.0,2024-03-10T19:59:38.000Z,NaN,['Si No Quieres No - Single'],NaN,...,2024-06-27T00:00:00.000Z,4566900,NaN,0.000000,9.0,6,2024-04-18T00:00:00.000Z,15,"[{'rank': 9, 'timestp': '2024-06-20T00:00:00.0...","[{'views': 5035818, 'timestp': '2024-06-20T00:..."
9,MgsdDfdGdHc,Si Antes Te Hubiera Conocido,USUG12400968,/g/11bt_1zp4n,https://i.ytimg.com/vi/MgsdDfdGdHc/maxresdefau...,123222923.0,2024-06-21T00:00:06.000Z,NaN,['Si Antes Te Hubiera Conocido'],['KAROL G'],...,2024-06-27T00:00:00.000Z,4554424,NaN,NaN,NaN,10,2024-06-27T00:00:00.000Z,1,"[{'rank': 10, 'timestp': '2024-06-27T00:00:00....","[{'views': 4554424, 'timestp': '2024-06-27T00:..."
